In [1]:
from model.ctabgan import CTABGAN
from model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics
import numpy as np
import pandas as pd
import glob

/Users/elismg/miniconda3/envs/CTAB-GAN-torch-update/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
pid = os.getpid()
print(f"Kernel PID: {pid}")

Kernel PID: 69287


In [6]:
num_exp = 1

dataset = "real_support2_small_Ostar_DP"
real_path = "Real_Datasets/real_support2_small_Ostar.csv"
fake_file_root = "Fake_Datasets"

real_data = pd.read_csv(real_path)

In [ ]:
synthesizer =  CTABGAN(private=False, # True,
                 test_ratio = 0.20,
                 categorical_columns = ['Y'], # any categorical column (binary, multi class)
                 log_columns = ['bun','crea', 'bili', 'wblc', 'dnrday', 'hday', 'totmcst', 'totcst', 'charges', 'slos'], # long tailed ditributions: assessed by empirical marginal histograms
                 mixed_columns= {}, # categorical and continuous data in one feature
                 general_columns = [], # only for simple, single mode continuous distributions or for categorical features that have computationally prohibitive many categories
                 non_categorical_columns = [], # no idea what this is
                 integer_columns = [],
                 problem_type= {"Classification": 'Y'}) 

for i in range(num_exp):
    synthesizer.fit(data=real_data)
    syn = synthesizer.generate_samples()
    
    syn.to_csv(f'{fake_file_root}/{dataset}/{dataset}_fake_{i}.csv', index= False)

In [13]:
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")

In [14]:
classifiers_list = ["lr","dt","rf","mlp"]
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",classifiers_list, test_ratio = 0.20)

result_df  = pd.DataFrame(result_mat,columns=["Acc","AUC","F1_Score"])
result_df.index = classifiers_list
result_df

,Acc,AUC,F1_Score
lr,NaN,NaN,NaN
dt,NaN,NaN,NaN
rf,NaN,NaN,NaN
mlp,NaN,NaN,NaN


In [ ]:
adult_categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_path,fake_path,adult_categorical)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

In [ ]:
priv_res_avg = []
for fake_path in fake_paths:
    priv_res = privacy_metrics(real_path,fake_path)
    priv_res_avg.append(priv_res)
    
privacy_columns = ["DCR between Real and Fake (5th perc)","DCR within Real(5th perc)","DCR within Fake (5th perc)","NNDR between Real and Fake (5th perc)","NNDR within Real (5th perc)","NNDR within Fake (5th perc)"]
privacy_results = pd.DataFrame(np.array(priv_res_avg).mean(axis=0).reshape(1,6),columns=privacy_columns)
privacy_results